In [1]:
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader
import scipy.stats
from geopy.geocoders import Nominatim
import geopandas as gp
from __future__ import print_function, division

try:
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib

import statsmodels.api as sm
import statsmodels.formula.api as smf


%pylab inline
import json
import os, shutil
import sys
import requests

Populating the interactive namespace from numpy and matplotlib


In [2]:
#data read-in
!wget https://stacks.stanford.edu/file/druid:py883nd2578/TX-clean.csv.gz
os.system("mv TX-clean.csv.gz " + os.getenv("PUIDATA")) #moves file to PUIDATA directory


--2018-02-18 13:26:02--  https://stacks.stanford.edu/file/druid:py883nd2578/TX-clean.csv.gz
Resolving stacks.stanford.edu... 171.67.37.91
Connecting to stacks.stanford.edu|171.67.37.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/gzip]
Saving to: “TX-clean.csv.gz”

    [    <=>                                ] 648,508,773  800K/s   in 13m 18s 

2018-02-18 13:39:20 (794 KB/s) - “TX-clean.csv.gz” saved [648508773]



0

In [3]:
## READ DATA INTO PANDAS DATAFRAME

TXdata = pd.read_csv(os.getenv("PUIDATA") + "/" +  "TX-clean.csv.gz", index_col = None, compression = 'gzip')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,16,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
TXdata.head(25)

,id,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,...,search_conducted,search_type_raw,search_type,contraband_found,stop_outcome,is_arrested,lat,lon,officer_id,driver_race_original
0,TX-2006-0000001,TX,2006-01-01,00:00,Walker,Walker County,48471.0,674,NaN,F,...,False,NaN,NaN,False,Warning,NaN,NaN,NaN,09592,White
1,TX-2006-0000002,TX,2006-01-01,00:00,Hansford,Hansford County,48195.0,50,NaN,M,...,False,NaN,NaN,False,Citation,NaN,NaN,NaN,05457,White
2,TX-2006-0000003,TX,2006-01-01,00:00,Montgomery,Montgomery County,48339.0,700,NaN,M,...,True,Probable Cause,Probable Cause,True,Citation,NaN,NaN,NaN,11193,Hispanic
3,TX-2006-0000004,TX,2006-01-01,00:00,Chambers,Chambers County,48071.0,819,NaN,M,...,False,NaN,NaN,False,Warning,NaN,NaN,NaN,11758,White
4,TX-2006-0000005,TX,2006-01-01,00:00,Montgomery,Montgomery County,48339.0,456,NaN,M,...,True,Probable Cause,Probable Cause,True,Citation,NaN,NaN,NaN,11560,White
5,TX-2006-0000006,TX,2006-01-01,00:00,Frio,Frio County,48163.0,104,NaN,M,...,False,NaN,NaN,False,Warning,NaN,NaN,NaN,11030,Hispanic
6,TX-2006-0000007,TX,2006-01-01,00:00,Castro,Castro County,48069.0,148,NaN,M,...,False,NaN,NaN,False,Warning,NaN,NaN,NaN,09449,Hispanic
7,TX-2006-0000008,TX,2006-01-01,00:00,Cameron,Cameron County,48061.0,554,NaN,M,...,False,NaN,NaN,False,Citation,NaN,NaN,NaN,10557,Hispanic
8,TX-2006-0000009,TX,2006-01-01,00:00,Eastland,Eastland County,48133.0,363,NaN,M,...,False,NaN,NaN,False,Citation,NaN,NaN,NaN,10866,Hispanic
9,TX-2006-0000010,TX,2006-01-01,00:00,Van Zandt,Van Zandt County,48467.0,516,NaN,M,...,False,NaN,NaN,False,Warning,NaN,NaN,NaN,10186,White


In [5]:
TXdata.shape

(23397249, 26)

In [6]:
TXdata.columns

Index([u'id', u'state', u'stop_date', u'stop_time', u'location_raw',
       u'county_name', u'county_fips', u'fine_grained_location',
       u'police_department', u'driver_gender', u'driver_age_raw',
       u'driver_age', u'driver_race_raw', u'driver_race', u'violation_raw',
       u'violation', u'search_conducted', u'search_type_raw', u'search_type',
       u'contraband_found', u'stop_outcome', u'is_arrested', u'lat', u'lon',
       u'officer_id', u'driver_race_original'],
      dtype='object')

In [7]:
TXdata.isnull().sum()

id                            0
state                         0
stop_date                     0
stop_time                     0
location_raw                694
county_name                 705
county_fips                 705
fine_grained_location       347
police_department             0
driver_gender              1712
driver_age_raw             1171
driver_age                 1286
driver_race_raw            3984
driver_race                4817
violation_raw              2178
violation                  2178
search_conducted              0
search_type_raw            2240
search_type              279866
contraband_found             34
stop_outcome               2325
is_arrested                   0
officer_id                   12
dtype: int64